### ALP spectral fitting

This notebook is fitting the ALP model to the simulated spectra. The first three cells are the same as in the previous notebook.  

In [1]:
# importing general packages

import numpy as np
import os, sys
import matplotlib.pyplot as plt

import xspec as xs
xs.Xset.allowPrompting = False #keeps pyxspec from hanging, waiting a response to a prompt
xs.Xset.allowNewAttributes = True

In [2]:
# The units for the model need to be given per bin. Number of bins determined by the 
# input LLE background spectrum.

from astropy.io import fits

hdu = fits.open("bn121225417_LAT-LLE_bkgspectra.bak")
energy = hdu[2].data # this holds energies corresponding to bin ends, in keV
E_MeV_diff = ((energy['E_MAX'] - energy['E_MIN']))/1000. # bin sizes, in MeV
nBins = len(E_MeV_diff)
# print E_MeV_diff

In [3]:
SED10 = np.load('bn121225417_SED10.npy') 

def ALP(engs, params, flux):
    norm1 = params[0]
    flux[:] = norm1 * SED10[:]*E_MeV_diff/nBins 
    # print(len(flux), len(engs)) # these two have to be the same
        
ALPInfo = ("norm1 \"\" 1.1  1e-12  1e-10 1e10 1e12 0.0001",)
xs.AllModels.addPyMod(ALP, ALPInfo, 'add', spectrumDependent=True)

In [4]:
# normalizations, coarse grid
N = np.logspace(np.log10(8.4e-8), np.log10(8.4e2), 30)

normALP = np.zeros((30,2001))
chi_stat = np.zeros((30,2001))
Fit_st = np.zeros((30,2001))

xs.AllData.clear()   # clear all data, if any.

for j in range(30):
    for i in range(1,2001):
        xs.AllData.clear()
        s=xs.Spectrum("bn121225417_ALP%s_bkg.fak{%s}" %(j,i)) 
        s.response = "bn121225417_LAT-LLE_weightedrsp.rsp"
        s.background= ("bn121225417_LAT-LLE_bkgspectra.bak{1}")
        xs.AllModels.clear()
        m=xs.Model("ALP")
        xs.Fit.statMethod = "pgstat"
        m.ALP.norm1.values=1.
        m.ALP.norm1.frozen = True
        #List of value floats [val,delta,min,bot,top,max].
        #m.ALP.norm.values=[N[j], N[j]*0.1, N[j]*0.1, N[j]*0.95, N[j]*1.05, N[j]*1.95]
        m.ALP.norm.frozen = False
        xs.Fit.perform()
        normALP[j,i] = m.ALP.norm.values[0]
        chi_stat[j,i] = xs.Fit.testStatistic
        Fit_st[j,i] = xs.Fit.statistic
        
np.save('TS_pgfit_ALP.npy', chi_stat)
np.save('pgfit_ALP.npy', Fit_st)
np.save('norm_ALP.npy', normALP)       